### Marcação POS, lematização e stemização

In [19]:
import bs4 as bs
import urllib.request
import nltk
import spacy

In [20]:
pln = spacy.load('pt_core_news_sm')
pln

In [6]:
documento = pln('Estou aprendendo processamento de linguagem natural, curso em Curitiba')

In [7]:
type(documento)

spacy.tokens.doc.Doc

In [8]:
# cada argumento no documento é um token
for token in documento:
    print(token.text, token.pos_)


Estou AUX
aprendendo VERB
processamento NOUN
de ADP
linguagem NOUN
natural ADJ
, PUNCT
curso NOUN
em ADP
Curitiba PROPN


In [9]:
for token in documento:
    print(token.text, token.lemma_)


Estou estar
aprendendo aprender
processamento processamento
de de
linguagem linguagem
natural natural
, ,
curso curso
em em
Curitiba Curitiba


In [12]:
doc = pln('encontrei encontraram encontrarão encontrariam cursando curso cursei')
[token.lemma_ for token in doc]

['encontrar',
 'encontrar',
 'encontrar',
 'encontrarer',
 'cursar',
 'curso',
 'cursei']

In [3]:
import nltk
nltk.download('rslp')

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\vitao\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [4]:
stemmer = nltk.stem.RSLPStemmer()
stemmer.stem('aprender')

'aprend'

In [15]:
for token in documento:
    print(token.text, token.lemma_, stemmer.stem(token.text))

Estou estar est
aprendendo aprender aprend
processamento processamento process
de de de
linguagem linguagem lingu
natural natural natur
, , ,
curso curso curs
em em em
Curitiba Curitiba curitib


### Carregamento da base de dados

In [5]:
dados = urllib.request.urlopen('https://pt.wikipedia.org/wiki/Intelig%C3%AAncia_artificial')

In [6]:
dados = dados.read()
#dados

In [8]:
dados_html = bs.BeautifulSoup(dados,'lxml')
dados_html

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="pt">
<head>
<meta charset="utf-8"/>
<title>Inteligência artificial – Wikipédia, a enciclopédia livre</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","janeiro","fevereiro","março","abril","maio","junho","julho","agosto","setembro","outubro","novembro","dezembro"],"wgRequestId":"1acbf686-9dfe-468a-aaac-203819663341","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Inteligência_artificial","wgTitle":"Inteligência artificial","wgCurRevisionId":63711508,"wgRevisionId":63711508,"wgArticleId":12722,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["!Páginas com erros em referências","!CS1 inglês-fontes em língua (en)","!CS1 manut: Nomes múltip

In [9]:
paragrafos = dados_html.find_all('p')

In [10]:
len(paragrafos)

114

In [11]:
paragrafos[0]

<p>
		Páginas para editores conectados <a aria-label="Saiba mais sobre edição" href="/wiki/Wikip%C3%A9dia:Boas-vindas"><span>saiba mais</span></a>
</p>

In [16]:
conteudo = ''
for p in paragrafos:
    conteudo += p.text

In [ ]:
conteudo = conteudo.lower()
conteudo

### Buscas em textos com spaCy

In [21]:
pln = spacy.load('pt_core_news_sm')
pln

In [23]:
string = 'turing'
token_pesquisa = pln(string)

In [24]:
from spacy.matcher import PhraseMatcher
matcher = PhraseMatcher(pln.vocab)
matcher.add('SEARCH', None, token_pesquisa)

In [28]:
# Encontrou a palavra turing 13x
doc = pln(conteudo)
matches = matcher(doc)
matches


[(8661325627334373315, 3122, 3123),
 (8661325627334373315, 3132, 3133),
 (8661325627334373315, 3288, 3289),
 (8661325627334373315, 3293, 3294),
 (8661325627334373315, 3299, 3300),
 (8661325627334373315, 3362, 3363),
 (8661325627334373315, 3989, 3990),
 (8661325627334373315, 5254, 5255),
 (8661325627334373315, 8705, 8706),
 (8661325627334373315, 8840, 8841),
 (8661325627334373315, 8935, 8936),
 (8661325627334373315, 9052, 9053)]

In [29]:
print('A palavra é:', doc[3122:3123])
print('frase:', doc[3122-5:3123+5])

A palavra é: turing
frase: uma contribuição prática de alan turing foi o que se chamou


In [35]:
from IPython.core.display import HTML
numero_palavras = 50
doc = pln(conteudo)
matches = matcher(doc)
texto = ''
display(HTML(f'<h1>{string.upper()}</h1>'))
display(HTML(f"""<p><strong>Resultados encontrados: </strong> {len(matches)}</p>"""))
for i in matches:
    inicio = i[1]-numero_palavras
    if inicio < 0:
        inicio = 0
    texto += str(doc[inicio:i[2] + numero_palavras]).replace(string,f"<mark>{string}</mark>")
    texto += "<br /> <br />"
display(HTML(f"""... {texto} ..."""))


### Extração de entidades nomeadas

In [ ]:
for entidade in doc.ents:
    print(entidade.text, entidade.label_)

In [37]:
from spacy import displacy
displacy.render(doc,style='ent', jupyter=True)